In [ ]:
import pandas as pd
import numpy as np
import requests
import re
import csv

### GGF HH production

In [ ]:
pythia_fragment_CP5 = """
from Configuration.Generator.Pythia8CommonSettings_cfi import *
from Configuration.Generator.MCTunes2017.PythiaCP5Settings_cfi import *
from Configuration.Generator.PSweightsPythia.PythiaPSweightsSettings_cfi import *

generator = cms.EDFilter("Pythia8ConcurrentHadronizerFilter",
                         maxEventsToPrint = cms.untracked.int32(1),
                         pythiaPylistVerbosity = cms.untracked.int32(1),
                         filterEfficiency = cms.untracked.double(1.0),
                         pythiaHepMCVerbosity = cms.untracked.bool(False),
                         comEnergy = cms.double(13000.),
                         PythiaParameters = cms.PSet(
        pythia8CommonSettingsBlock,
        pythia8CP5SettingsBlock,
        pythia8PSweightsSettingsBlock,
        processParameters = cms.vstring(__CHANNEL_DECAY_FRAGMENT__),
        parameterSets = cms.vstring('pythia8CommonSettings',
                                    'pythia8CP5Settings',
                                    'pythia8PSweightsSettings',
                                    'processParameters'
                                    )
        )
                         )


ProductionFilterSequence = cms.Sequence(generator)
"""

In [ ]:
def fragmentsDictCreator (decay_fr):
    dict = { '2016': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2016APV': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2017': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr),
             '2018': pythia_fragment_CP5.replace('__CHANNEL_DECAY_FRAGMENT__',decay_fr)
                    }
    return dict

In [ ]:
LHEproducer = """
import FWCore.ParameterSet.Config as cms

# link to cards:
# __EXAMPLE__

externalLHEProducer = cms.EDProducer('ExternalLHEProducer',
    args = cms.vstring('__GRIDPACK__'),
    nEvents = cms.untracked.uint32(5000),
    numberOfParameters = cms.uint32(1),
    outputFile = cms.string('cmsgrid_final.lhe'),
    generateConcurrently = cms.untracked.bool(True),
    scriptName = cms.FileInPath('GeneratorInterface/LHEInterface/data/run_generic_tarball_cvmfs.sh')
)
"""

In [ ]:
years = ["2016", "2016APV", "2017", "2018"]
processes = ["Radion"]#,"BulkGraviton"]
mass_point = ["1100","1200","1300","1400","1600","1700","1800","1900","2200","2400","2600","2800"]
tot_events = [400000]*12
gridpacks_dict = {}
example_dict = {}
dataset_names = {}

In [ ]:
for year in range(len(years)):
    t_example1 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/exo_diboson/Spin-0'
    t_example2 = 'https://github.com/cms-sw/genproductions/tree/master/bin/MadGraph5_aMCatNLO/cards/production/2017/13TeV/exo_diboson/Spin-2'
    t_datasetname_year = 'GluGluTo{process}ToHHTo2B2ZTo2L2J_M-{mass}_narrow_TuneCP5_PSWeights_13TeV-madgraph-pythia8'
    t_gp_year = "/cvmfs/cms.cern.ch/phys_generator/gridpacks/UL/13TeV/madgraph/V5_2.6.5/GF_Spin_{spin}/{process}_hh{graviton}narrow_M{mass}/v1/{process}_hh{graviton}narrow_M{mass}_slc7_amd64_gcc700_CMSSW_10_6_19_tarball.tar.xz"    
    tmp_dataset_dict={}
    tmp_gridpack_dict={}
    tmp_example_dict={}
    dataset_names_year = []
    gp_ggf_year = "gp_ggf_" + years[year]
    gp_ggf_year = []
    

    for process in range(len(processes)):
        #gp_ggf_year = "gp_ggf_" + years[year]
        dataset_names_year = []
        gp_ggf_year = []

        for mass in mass_point:
            if processes[process] == "Radion":
                d = {"process":processes[process], "mass": mass, "spin": "0", "graviton": "_"}
                tmp_example_dict[processes[process]] = t_example1
            elif processes[process] == "BulkGraviton":
                d = {"process":processes[process], "mass": mass, "spin": "2", "graviton": "_GF_HH_"}
                tmp_example_dict[processes[process]] = t_example2
            gp_ggf_year.append(t_gp_year.format_map(d))
            dataset_names_year.append(t_datasetname_year.format_map(d))
            tmp_dataset_dict[processes[process]] = dataset_names_year
            tmp_gridpack_dict[processes[process]] = gp_ggf_year
    gridpacks_dict[years[year]] = tmp_gridpack_dict
    dataset_names[years[year]] = tmp_dataset_dict
    example_dict[years[year]] = tmp_example_dict

#print(example_dict['2016']['Radion'])
#gridpacks_dict = {'VBF_HH' : {'2016': gp_vbf_2016, '2017': gp_vbf_2017, '2018': gp_vbf_2018}}                                                                                                              
#for x in gridpacks_dict:                                                                                                                                                                                   
#    print (x)                                                                                                                                                                                              
#    for y in gridpacks_dict[x]:                                                                                                                                                                            
#        print (y,':', gridpacks_dict[x][y])                 

In [ ]:
ggf_hh_bbzz = '''                                                                                                                                                                                           
    '15:onMode = off',
    '15:onIfAny = 11 13', # only leptonic tau decays
    '23:mMin = 0.05',
    '23:onMode = off',
    '23:onIfAny = 1 2 3 4 5 11 13 15', # Z->jets decay and a leptonic charged Z decay, including taus
    '24:mMin = 0.05',
    '24:onMode = off',
    '25:m0 = 125.0',
    '25:onMode = off',
    '25:onIfMatch = 5 -5',
    '25:onIfMatch = 23 23',
    'ResonanceDecayFilter:filter = on',
    'ResonanceDecayFilter:exclusive = on', #off: require at least the specified number of daughters, on: require exactly the specified number of daughters
    'ResonanceDecayFilter:eMuAsEquivalent = off', #on: treat electrons and muons as equivalent
    'ResonanceDecayFilter:eMuTauAsEquivalent = on', #on: treat electrons, muons , and taus as equivalent
    'ResonanceDecayFilter:allNuAsEquivalent  = off', #on: treat all three neutrino flavours as equivalent
    'ResonanceDecayFilter:udscAsEquivalent   = off', #on: treat udsc quarks as equivalent
    'ResonanceDecayFilter:udscbAsEquivalent  = on',  #on: treat udscb quarks as equivalent
    'ResonanceDecayFilter:mothers = 25,23', #list of mothers not specified -> count all particles in hard process+resonance decays (better to avoid specifying mothers when including leptons from the lhe in counting, since intermediate resonances are not gauranteed to appear in general
    'ResonanceDecayFilter:daughters = 5,5,23,23,1,1,11,11',
'''
#'SpaceShower:dipoleRecoil = on' 


## .csv Production

In [ ]:
#MadgraphVersion = re.compile("V5_2\.[0-9]\.[0-9]")
process_pythia_map = {}
tmp_process_pythia_map = {}
for process in processes:
    tmp_process_pythia_map[process] = fragmentsDictCreator(ggf_hh_bbzz)
process_pythia_map.update(tmp_process_pythia_map)

In [ ]:
#for year in range(len(years)):
for year in gridpacks_dict:
    #print(year)
    for process in processes:
        with open('ggF_'+process+'_'+year+'_new.csv', 'w') as csvfile:
            csvwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csvwriter.writerow(['Dataset name','Events','fragment','notes','Generator','mcdbid','time','size'])
            for i in range(len(mass_point)):
                tmp_fragment=""
                #print(gridpacks_dict[year][i])
                #print(dataset_names[year][i])
                #version = MadgraphVersion.search(path).group(0)
                version="2.6.5"
                dataset_name = dataset_names[year][process][i]
                if year == "2016APV":
                    events = round(tot_events[i]*0.54)
                elif year == "2016":
                    events = round(tot_events[i]*0.46)
                elif year == "2017":
                    events = tot_events[i]
                elif year == "2018":
                    events = tot_events[i]
                    
                tmp_fragment = LHEproducer.replace('__GRIDPACK__',gridpacks_dict[year][process][i]) + '\n' + process_pythia_map[process][year]
                final_fragment = tmp_fragment.replace('__EXAMPLE__',example_dict[year][process])
                note = dataset_name.replace('_',' ')
                generators="Madgraph_" + version + "  Pythia8"
                mcdb_id = '0'
                time = '0.3'
                size = '120'                   
                csvwriter.writerow([dataset_name, events, final_fragment, note, generators, mcdb_id, time, size])